In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
import re

In [50]:
import tqdm

In [2]:
import os

os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

INFO:tensorflow:Initializing the TPU system: grpc://10.71.33.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.71.33.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Using TPU


In [3]:
!pip install -q tensorflow-text

     |████████████████████████████████| 3.4MB 7.0MB/s 


In [4]:
import random

In [5]:
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
data = pd.read_csv('/content/drive/MyDrive/Abbreviation_detection_dataset/archive/full_data.csv', nrows = 10000)
data.head()

,TEXT,LOCATION,LABEL
0,alphabisabolol has a primary antipeptic action...,56,substrate
1,a report is given on the recent discovery of o...,24|49|68|113|137|172,carcinosarcoma|recovery|reference|recovery|aft...
2,the virostatic compound nndiethyloxotetradecyl...,55,substrate
3,rmi rmi and rmi are newly synthetized nrdibenz...,25|82|127|182|222,compounds|compounds|inhibitory|lethal doses|ca...
4,a doubleblind study with intraindividual compa...,22|26|28|77|90|144|158|203,oxazepam|placebo|oral administration|pentagast...


In [10]:
sentences = []
abber_locs = []

for i in range(0, len(data)):
    sentences.append(data['TEXT'][i])
    abber_locs.append(data['LOCATION'][i].split("|"))

In [11]:
abbers = []

for i in range(0, len(abber_locs)):
    temp = []
    for l in abber_locs[i]:
        temp.append(sentences[i].split()[int(l)])
    abbers.append(temp)

In [12]:
def text_cleaner(text):
    new_string = text.lower()
    new_string = BeautifulSoup(new_string, 'lxml').text
    new_string = re.sub("[^a-zA-Z0-9]", " ", new_string)
    tokens = [w for w in new_string.split() if not w in stop_words]
    return (' '.join(tokens)).strip()

def abbreviation_cleaner(text):
    new_string = text.lower()
    new_string = re.sub("[^a-zA-Z0-9]", "", new_string)
    return new_string

In [13]:
cleaned_sentences = []
for i in range(0, len(sentences)):
    cleaned_sentences.append(text_cleaner(sentences[i]))

cleaned_abbers = []
for i in abbers:
    temp = []
    for j in i:
        temp.append(abbreviation_cleaner(j))
    cleaned_abbers.append(temp)

In [16]:
print("total no of abberviations:", sum([len(x) for x in cleaned_abbers]))

total no of abberviations: 29916


In [17]:
def select_words(sentence, abber_list):
  j = 0
  words = sentence.split()
  random.shuffle(words)
  temp = []
  for i in range(0, len(words)):
    if (words[i] in abber_list):
      continue
    else:
      temp.append(words[i])
      j += 1
    if (j == len(abber_list)):
      break
  return temp

In [19]:
normal_words = []
for i in range(0, len(cleaned_sentences)):
  normal_words.append(select_words(cleaned_sentences[i], cleaned_abbers[i]))

In [23]:
sentences = []
words = []
labels = []
for i in range(0, len(cleaned_sentences)):
  for j in range(0, len(cleaned_abbers[i])):
    sentences.append(cleaned_sentences[i])
    words.append(cleaned_abbers[i][j])
    labels.append(1)
    sentences.append(cleaned_sentences[i])
    words.append(normal_words[i][j])
    labels.append(0)

In [25]:
char2idx = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15,
            'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '1': 27, '2': 28, '3': 29,
            '4': 30, '5': 31, '6': 32, '7': 33, '8': 34, '9': 35, '0': 36}

In [26]:
preprocess_lnk = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess = hub.load(preprocess_lnk)

In [27]:
tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs, arguments=dict(seq_length=400))

In [28]:
enc_lnk = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'
bert_encoder = hub.KerasLayer(enc_lnk, name='encoder')

In [47]:
def find_current_word_embedding(word, sentence):
  tokens = tokenizer([word]).to_list()[0][0]
  tokens2 = tokenizer([sentence])
  tokens2list = tokens2[0].to_list()
  j = 1
  for i in range(0, len(tokens2list)):
    if (tokens2list[i] == tokens):
      break
    else:
      j += len(tokens2list[i])
  k = j + len(tokens)
  segs = packer([tokens2])
  out = bert_encoder(segs)
  return out['sequence_output'][0][j:k].numpy()

In [40]:
list1 = ['apples', 'bananas', 'oranges', 'mangoes', 'papaya']
list2 = ['lithium', 'boron', 'oxygen', 'carbon', 'nitrogen']
##random.seed(10)
random.Random(15).shuffle(list1)
random.Random(15).shuffle(list2)

In [41]:
print(list1, list2)

['papaya', 'mangoes', 'oranges', 'apples', 'bananas'] ['nitrogen', 'carbon', 'oxygen', 'lithium', 'boron']


In [42]:
random.Random(10).shuffle(sentences)
random.Random(10).shuffle(words)
random.Random(10).shuffle(labels)

In [48]:
train_sentences = sentences[:10000]
train_words = words[:10000]
train_labels = labels[:10000]

In [71]:
val_sentences = sentences[10000:12000]
val_words = words[10000:12000]
val_labels = labels[10000:12000]

In [45]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [46]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [51]:
train_berts = []
train_tokens = []

for i in tqdm.notebook.tqdm(range(0, len(train_sentences))):
  train_berts.append(find_current_word_embedding(train_words[i], train_sentences[i]))
  train_tokens.append([char2idx[x] for x in train_words[i]])

In [63]:
val_berts = []
val_tokens = []

for i in tqdm.notebook.tqdm(range(0, len(val_sentences))):
  val_berts.append(find_current_word_embedding(val_words[i], val_sentences[i]))
  val_tokens.append([char2idx[x] for x in val_words[i]])

In [52]:
from keras.preprocessing.sequence import pad_sequences
train_tokens_enc = pad_sequences(train_tokens, maxlen = 10, padding = 'post')

In [68]:
type(train_tokens_enc)

numpy.ndarray

In [64]:
val_tokens_enc = pad_sequences(val_tokens, maxlen = 10, padding = 'post')

In [53]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [54]:
## Model

input1 = layers.Input(shape = (None, 128))
lstm1 = layers.LSTM(3)
out1 = lstm1(input1)

input2 = layers.Input(shape = (10,))
enc_emb = layers.Embedding(37, 10, trainable = True)(input2)
lstm2 = layers.LSTM(3)
out2 = lstm2(enc_emb)

final_input = layers.Concatenate(axis = -1, name = 'concat_layer')([out1, out2])
dense_last = layers.Dense(1)
final_output = dense_last(final_input)

In [55]:
model = Model([input1, input2], final_output)

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 10)       370         input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 3)            1584        input_1[0][0]                    
______________________________________________________________________________________________

In [57]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [58]:
for i in range(0, len(train_berts)):
  if (len(train_berts[i]) > 5):
    train_berts[i] = train_berts[i][:5]
  elif (len(train_berts[i]) == 5):
    continue
  else:
    train_berts[i] = np.concatenate((train_berts[i], np.zeros((5-len(train_berts[i]), 128))))

In [65]:
for i in range(0, len(val_berts)):
  if (len(val_berts[i]) > 5):
    val_berts[i] = val_berts[i][:5]
  elif (len(val_berts[i]) == 5):
    continue
  else:
    val_berts[i] = np.concatenate((val_berts[i], np.zeros((5-len(val_berts[i]), 128))))

In [ ]:
train_berts = np.array(train_berts)
val_berts = np.array(val_berts)
train_labels = np.array(train_labels)
val_labels = np.arrya(val_labels)

In [78]:
model.fit(x = [train_berts, train_tokens_enc], y = train_labels, epochs = 10, validation_data = ([val_berts, val_tokens_enc], val_labels)) 

Epoch 1/10
313/313 [==============================] - 6s 17ms/step - loss: 0.2066 - accuracy: 0.9386 - val_loss: 0.1925 - val_accuracy: 0.9440
Epoch 2/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1915 - accuracy: 0.9403 - val_loss: 0.1815 - val_accuracy: 0.9455
Epoch 3/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1832 - accuracy: 0.9415 - val_loss: 0.1839 - val_accuracy: 0.9465
Epoch 4/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1731 - accuracy: 0.9433 - val_loss: 0.1731 - val_accuracy: 0.9465
Epoch 5/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1628 - accuracy: 0.9462 - val_loss: 0.1664 - val_accuracy: 0.9525
Epoch 6/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1572 - accuracy: 0.9485 - val_loss: 0.1685 - val_accuracy: 0.9535
Epoch 7/10
313/313 [==============================] - 5s 15ms/step - loss: 0.1924 - accuracy: 0.9323 - val_loss: 0.1928 - val_accuracy: 0.9450